# 5章　閉ループ系に注目した制御系設計

## 5.6　状態フィードバック制御

ライブラリの読み込みと関数の定義

※以下のpipは初回のみ実施ください。

In [ ]:
%pip install sympy
%pip install control
%pip install numpy scipy control

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.family'] ='sans-serif' #使用するフォント
plt.rcParams['xtick.direction'] = 'in' #x軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams['ytick.direction'] = 'in' #y軸の目盛線が内向き('in')か外向き('out')か双方向か('inout')
plt.rcParams['xtick.major.width'] = 1.0 #x軸主目盛り線の線幅
plt.rcParams['ytick.major.width'] = 1.0 #y軸主目盛り線の線幅
plt.rcParams['font.size'] = 10 #フォントの大きさ
plt.rcParams['axes.linewidth'] = 1.0 # 軸の線幅edge linewidth。囲みの太さ
plt.rcParams['mathtext.default'] = 'regular'
plt.rcParams['axes.xmargin'] = '0' #'.05'
plt.rcParams['axes.ymargin'] = '0.05'
plt.rcParams['savefig.facecolor'] = 'None'
plt.rcParams['savefig.edgecolor'] = 'None'

In [ ]:
# 線種を変更するジェネレータ
def linestyle_generator():
    linestyle = ['-', '--', '-.', ':']
    lineID = 0
    while True:
        yield linestyle[lineID]
        lineID = (lineID + 1) % len(linestyle)

# 時間応答のグラフを整える関数
def plot_set(fig_ax, *args):
    fig_ax.set_xlabel(args[0])
    fig_ax.set_ylabel(args[1])
    fig_ax.grid(ls=':')
    if len(args)==3:
        fig_ax.legend(loc=args[2])

# ボード線図を整える関数
def bodeplot_set(fig_ax, *args):
    fig_ax[0].grid(which='both', ls=':')
    fig_ax[0].set_ylabel('Gain [dB]')

    fig_ax[1].grid(which='both', ls=':')
    fig_ax[1].set_xlabel('$\\omega$ [rad/s]')
    fig_ax[1].set_ylabel('Phase [deg]')

    if len(args) > 0:
        fig_ax[1].legend(loc=args[0])
    if len(args) > 1:
        fig_ax[0].legend(loc=args[1])

# 状態フィードバック制御　p.177

システムの「内部の状態」をすべて観測し、その情報を使ってシステムを思い通りに操る制御手法

状態空間モデル $\dot{x} = Ax + Bu$ で記述されたシステムに対して制御器を設計していく。状態空間モデルには、入力と出力に加えて、状態があった。ここでは、この状態がセンサなどを用いてすべて観測可能であるとし、その観測された情報を用いて制御入力を決定することを考える。具体的には、

#### $u = Fx \quad (5.42)$

という状態フィードバック制御を考える。図にしたものが図 5.29 で、状態 $x$ の情報を利用して、制御入力 $u$ を決めるものとなっている。ここで、$F$ は、**状態フィードバックゲイン**と呼ばれる。

<img src="png/1.png" alt="1" width="300" height="150">

状態フィードバック制御は、PD制御とみることもできる。たとえば、台車系では、状態として、台車の位置 $z$ と速度 $\dot{z}$ をとることができる。ここで、状態ベクトル $x$ とフィードバックゲイン $F$ を成分で表すと以下のようになる。

$x = \begin{bmatrix} z \\ \dot{z} \end{bmatrix}, \quad F = \begin{bmatrix} f_1 & f_2 \end{bmatrix}$

このとき、式 (5.42) $u = Fx$ の行列演算を具体的に計算すると、次のようになる。

$\begin{aligned}
u &= \begin{bmatrix} f_1 & f_2 \end{bmatrix} \begin{bmatrix} z \\ \dot{z} \end{bmatrix} \\
&= f_1 \cdot z + f_2 \cdot \dot{z}
\end{aligned}$

したがって、式 (5.43) は、

#### $u = Fx = f_1 z + f_2 \dot{z} \quad (5.43)$

のように書くことができる。これは、PD制御である（$f_1$ が比例ゲインで $f_2$ が微分ゲイン）。

また、状態フィードバック制御ではフィードバックゲイン $F$ を設計することになるが、その代表的な方法として「**極配置法**」と「**最適レギュレータ**」がある。

## 5.6.1　極配置法　p.178

状態フィードバック制御において、閉ループ系の極（固有値）を、意図した配置にすることで、システムの挙動（安定性や応答速度）を設計する手法

システム $\dot{x} = Ax + Bu$ に状態フィードバック制御 $u = Fx$ を施すと、閉ループ系は、

#### $\dot{x} = (A + BF)x \quad (5.44)$

と記述することができる。元のシステム式に、制御入力の式を代入して整理する。

$\begin{aligned}
\dot{x} &= Ax + Bu & \text{（システムの方程式）} \\
&= Ax + B(Fx) & \text{（} u = Fx \text{ を代入）} \\
&= (A + BF)x & \text{（} x \text{ でくくる）}
\end{aligned}$

これにより、制御後のシステム全体の挙動は、行列 $A+BF$ によって決まることがわかる。行列 $A + BF$ のすべての固有値の実部が負であれば、システムが安定となる。したがって、そうなるように $F$ を設計する。

極配置法では、まず、$A + BF$ の固有値を指定する。具体的には、実部が負の固有値を状態の数だけ用意する。

つぎに、$A + BF$ の固有値が指定した固有値になるような $F$ を求める。

Python にはアッカーマンの極配置アルゴリズムが実装された便利な関数 acker が用意されているため、それを利用する。

### 「 F = - acker(A, B, p) 」

関数 acker の引数は、$A, B$, そして指定する閉ループ極。返り値が $A - BF$ の固有値が指定極になるような $F$ となるので、ここでは、負号を付けたものを $F$ としている。これにより、$A + BF$ の固有値が指定極になる $F$ が得られる。

実際、リスト 5.16 を実行すると、$F$ の値は、$F = [3 \quad -7]$ となる。そして、np.linalg.eigvals(P.A + P.B @ F) で固有値を確かめてみると、array([-1., -1.]) となるので、極配置ができていることがわかる。なお、@ は行列の積を計算する演算子のこと。

### リスト5.16　極配置

In [ ]:
from control.matlab import ss, acker, initial, lqr, care

A = [[0, 1], [-4, 5]]
B = [[0], [1]]
C = [[1, 0], [0, 1]]
D = [[0], [0]]
P = ss(A, B, C, D)

# A行列の固有値は不安定
eig_A = np.linalg.eigvals(P.A)

# 極配置で設計する。A+BFの固有値が指摘極polesになるように，ackerの結果に負号をつける
regulator_poles = [-1, -1]
F = -acker(P.A, P.B, regulator_poles)

F = np.atleast_2d(F) #値にかかわらず、かならず２次元配列にする

print("****状態空間モデル A B C D**** ")
print(P)
print("****A行列の固有値（負のため不安定）**** ")
print(eig_A)
print("****配置したい極の値**** ")
print(regulator_poles)
print("****極配置に対応したフィードバックゲインF**** ")
print(F)

#### （挙動の確認１） $F=0$ すなわち 入力 $u=0$ のときの挙動
まずフィードバックゲイン $F=0$ のときの動きを観察する。
リスト5.17のフィードバックゲインを F=[[0.,0.]] （初期設定値）で実行し、挙動を確認する。　初期状態は $x(0) = [-0.3 \quad 0.4]^\top$ と設定されている。入力 $u=0$ で状態 $x$ が発散し、不安定な挙動を示すことがわかる。
#### （挙動の確認２） $F$ を前述の極配置で求めた値に設定したときの挙動　
 $F$ を前述の極配置で求めた値に設定して閉ループ系 $\dot{x} = (A + BF)x$ の振る舞いを確認する。Fの値を極配置で求めた値に変更してリスト 5.17 を実行する。これより、状態 $x$ が $0$ に収束していることが確認できる。

### リスト5.17　状態フィードバック制御　図5.30　状態フィードバック制御(極配置法)

In [ ]:
#フィードバックゲイン　Ｆ　（初期設定値は０　あとで極配置のFに変えてください）
F=[[0.,0.]]

# 閉ループ系の A 行列を計算（F は状態フィードバックゲイン）
Acl = P.A + P.B @ F  # Matrix型なら @ を使用（*だと要素積になってしまう）

# 閉ループシステムの伝達関数（状態空間モデル）を作成
Pfb = ss(Acl, P.B, P.C, P.D)

# シミュレーション時間軸（0秒～5秒まで、0.01秒刻み）
Td = np.arange(0, 5, 0.01)
# 初期状態ベクトル x(0) = [-0.3, 0.4]
X0 = [-0.3, 0.4]
# ゼロ入力応答（u(t) = 0 のときの状態の時間変化）
x, t = initial(Pfb, Td, X0)

# プロット準備（サイズは 3 x 2.3 インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))
# 状態 x1 の時間変化をプロット
ax.plot(t, x[:, 0], label='$x_1$')
# 状態 x2 の時間変化を破線でプロット
ax.plot(t, x[:, 1], ls='-.', label='$x_2$')

# 軸ラベル・凡例などの設定（plot_set は自作関数だと思われます）
plot_set(ax, 't', 'x', 'best')

## 5.6.2　最適レギュレータ　p.182

「速く収束させたい」vs「エネルギー（入力）を節約したい」のバランスを重み行列 $Q, R$ で指定し、計算でベストな $F$ を求める方法

極配置法で状態フィードバックゲインを設計できることがわかった。しかし、

1. 固有値の実部を負側に大きくすると応答が速くなるが、フィードバックゲイン $F$ が大きくなり入力が大きくなる。

2. 状態変数の中に振れ幅が大きいものが現れることがある。

といった問題点があり、極の選定に苦労することがある。そこで、システムの動特性や入力エネルギーに関する評価指標を設定して、その値を最小化する状態フィードバックゲインを求める問題を考える。これは **最適制御問題（LQ最適制御問題）** と呼ばれている。

---------------------------------------------------------------------------------------------------------------------------------------------------

$Q=Q^\top > 0, \ R=R^\top > 0$ に対して、2次形式の評価関数

$J = \int_{0}^{\infty} (x(t)^\top Q x(t) + u(t)^\top R u(t)) dt \quad (5.46)$

を最小化するコントローラは、$u = F_{\text{opt}} x$ の形で得られ、$F_{\text{opt}}$ の値は

$F_{\text{opt}} = -R^{-1}B^\top P \quad (5.47)$となる。ただし、$P = P^\top > 0$ は、リカッチ方程式

$A^\top P + PA - PBR^{-1}B^\top P + Q = 0 \quad (5.48)$

を満たす唯一の正定対称解である。また、$J$ の最小値は、$x(0)^\top P x(0)$ である。

---------------------------------------------------------------------------------------------------------------------------------------------------

このような評価関数の最適化によって得られる状態フィードバック制御のことを **最適レギュレータ** という。最適制御問題では、評価関数 $J$ を最小化する初期時刻から無限時間先までの入力 $u$ を求めるが、その最適制御入力が、状態フィードバック制御則の形で与えられていることがポイント。なお、$Q$ は対角行列に設定することがほとんど。たとえば、

#### $Q = \begin{bmatrix} q_1 & 0 \\ 0 & q_2 \end{bmatrix} \quad (5.49)$

とすると、

#### $x(t)^\top Q x(t) = q_1 x_1(t)^2 + q_2 x_2(t)^2 \quad (5.50)$

となるので、$x_1$ を $x_2$ より速く $0$ に収束させたい場合には、$q_1 > q_2$ となるように設定する。一方、$R$ は、$u$ が 1 入力の場合には、スカラーになる。$R$ を大きく設定すると、入力が大きくなりすぎないフィードバックゲイン $F$ が得られることが期待できる。

※ (5.49)と(5.50)の経過が気になる方は「1．ベクトルの定義」を確認

最適レギュレータは、Python では、lqr を用いて設計することができる。

#### 「 F, X, E = lqr(A, B, Q, R) 」
#### 「 F = -F 」

引数の A と B は、システムの $A$ 行列と $B$ 行列であり、Q と R は重み行列 $Q$ と $R$ 。また、返り値の F はフィードバックゲイン、X はリカッチ方程式の解、E は閉ループ系の極（$A-BF$ の固有値）。lqr では、$u(t) = -Fx(t)$ に対する $F$ を求めるようになっているので、F = -F としています。これにより、$A+BF$ が安定となる。

####    

### 1. ベクトルの定義

まず、状態ベクトル $x(t)$ とその転置 $x(t)^\top$ を成分で書く。

$x(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}, \quad
x(t)^\top = \begin{bmatrix} x_1(t) & x_2(t) \end{bmatrix}$

### 2. 行列の掛け算のセット

式 (5.50) の左辺 $x(t)^\top Q x(t)$ に、定義した成分と $Q$ を代入する。

$x(t)^\top Q x(t) = 
\underbrace{\begin{bmatrix} x_1(t) & x_2(t) \end{bmatrix}}_{x(t)^\top}
\underbrace{\begin{bmatrix} q_1 & 0 \\ 0 & q_2 \end{bmatrix}}_{Q}
\underbrace{\begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}}_{x(t)}$

### 3. 計算ステップ

行列の掛け算は結合法則が成り立つので、後ろの2つ（$Q$ と $x(t)$）を先に計算する。

#### ステップ A: $Q \times x(t)$ の計算

$\begin{aligned}
Q x(t) &= \begin{bmatrix} q_1 & 0 \\ 0 & q_2 \end{bmatrix} \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix} \\
&= \begin{bmatrix} q_1 \cdot x_1(t) + 0 \cdot x_2(t) \\ 0 \cdot x_1(t) + q_2 \cdot x_2(t) \end{bmatrix} \\
&= \begin{bmatrix} q_1 x_1(t) \\ q_2 x_2(t) \end{bmatrix}
\end{aligned}$

#### ステップ B: $x(t)^\top \times (\text{ステップAの結果})$ の計算

これに、左側の $x(t)^\top$ を掛ける。

$\begin{aligned}
x(t)^\top (Q x(t)) &= \begin{bmatrix} x_1(t) & x_2(t) \end{bmatrix} \begin{bmatrix} q_1 x_1(t) \\ q_2 x_2(t) \end{bmatrix} \\
&= x_1(t) \cdot (q_1 x_1(t)) + x_2(t) \cdot (q_2 x_2(t)) \\
&= q_1 x_1(t)^2 + q_2 x_2(t)^2
\end{aligned}$

これによって、式 (5.50) が導かれる。

In [ ]:
# 状態重み行列 Q と 入力重み R の設定
Q = np.diag([100, 1])  # x1（例：位置）に高い重みを置く → 素早く制御したい
R = 1                 # 入力（制御力）の重み

# LQRによる最適フィードバックゲイン F、リカッチ行列 X、閉ループ極 E を計算
F, X, E = lqr(P.A, P.B, Q, R)

# Pythonの control.lqr() は u = -F x の形で出てくるので、F にマイナスをかけて慣例に合わせる
F = -F

print('--- フィードバックゲインF（lqrの計算結果） ---')
print(F)

# 別表現：F = -(1/R) * B^T * X を手計算で確認（理論式との一致チェック）
print('--- （理論の確認）フィードバックゲインF（lqrの計算結果から求まるリカッチ行列 XからFを計算⇒上述のFと一致している。） ---')
print(-(1/R) * P.B.T @ X)

# --- 閉ループ極（固有値）の表示 ---
print('--- 閉ループ極（lqrの計算結果） ---')
print(E)  # LQR設計による極（固有値）

# 確認：Acl = A + B F による閉ループ行列の固有値
print('--- （理論の確認）閉ループ極（Acl = A + B F による閉ループ行列の固有値）⇒上述の極と一致している。 ---')
print(np.linalg.eigvals(P.A + P.B @ F))

##### LQRによる状態フィードバック制御の挙動確認
LQRで求めたフィードバックゲインでの挙動を確認する。図５．３２の下にあるプログラムを実行して図を表示してください。

### 図5.32　状態フィードバック制御(最適レギュレータ)

In [ ]:
# 閉ループ系のA行列を定義（FはLQRで得た最適フィードバックゲイン）
Acl = P.A + P.B @ F

# 状態空間モデルで閉ループ系のシステムを定義
Pfb = ss(Acl, P.B, P.C, P.D)

# シミュレーション用の時間軸（0〜5秒を0.01秒刻みで）
tdata = np.arange(0, 5, 0.01)

# 初期状態 [-0.3, 0.4] からのゼロ入力応答（u(t)=0）を計算
xini, tini = initial(Pfb, tdata, [-0.3, 0.4])

# プロットの準備（サイズ 3 x 2.3 インチ）
fig, ax = plt.subplots(figsize=(3, 2.3))

# 状態 x1 の時間変化をプロット（実線）
ax.plot(tini, xini[:, 0], label='$x_1$')

# 状態 x2 の時間変化をプロット（点線）
ax.plot(tini, xini[:, 1], ls='-.', label='$x_2$')

# 軸ラベルとグリッド、凡例の設定
ax.set_xlabel('t')       # 時間軸のラベル
ax.set_ylabel('x')       # 状態変数のラベル
ax.grid(ls=':')          # 点線のグリッドを表示
ax.legend()              # 凡例を表示

####

## ハミルトン行列

「最適制御の問題（LQR）」を「極配置の問題」に変換してくれる橋渡し役

なお、リカッチ方程式の解を用いて、-(1/R) * (P.B.T) @ X を計算すればフィードバックゲイン $F_{\text{opt}}$ が求まる。

また、最適レギュレータのフィードバックゲイン $F_{\text{opt}}$ を用いると、$A + BF_{\text{opt}}$ の固有値は安定になるが、その固有値は、ハミルトン行列

$H = \begin{bmatrix} A & -BR^{-1}B^\top \\ -Q & -A^\top \end{bmatrix} \quad (5.51)$

の安定な固有値に等しいことが知られている。

したがって、リカッチ方程式を解かなくても、$H$ の安定固有値を求めて極配置法で $F_{\text{opt}}$ を求めることで同じ結果を得ることができる（リスト 5.18）。

### リスト5.18　ハミルトン行列の固有値を求める

In [ ]:
# ハミルトン行列 H の構築
# 上半分：A, -BR^{-1}B^T / 下半分：-Q, -A^T
H = np.block([[P.A, -P.B * (1 / R) @ P.B.T], [-Q, -P.A.T]])

# ハミルトン行列の固有値を計算
eigH = np.linalg.eigvals(H)
print(eigH)

# --- ハミルトン行列の安定固有値（実部が負のもの）を抽出 ---
print('--- ハミルトン行列の安定固有値 ---')
eigH_stable = [i for i in eigH if i.real < 0]  # 実部が負の固有値
print(eigH_stable)

# 安定固有値を極として使い、acker法（極配置法）でフィードバックゲインを設計
F = -acker(P.A, P.B, eigH_stable)

print('--- フィードバックゲイン ---')
print(F)

## 可制御性，可観測性 p.189~

### 可制御性

システムの状態xを、ある有限時間内に任意の初期状態から任意の最終状態へ制御入力 u(t)を使って到達できるかどうかを示す性質。

### 可観測性

システムの出力y(t)を使って、初期状態x(0)を一意に推定できるかどうかを示す性質。

### 可制御性

状態フィードバックゲインの極配置による設計において、任意の指定極で極配置を行うためには、システムが可制御である必要がある。また、システムが可制御であれば、最適レギュレータで閉ループ系を安定化することができる。このように、可制御性は、コントローラ設計が自由にできるかどうかを事前に把握するための重要な性質。

システム $\dot{x}(t) = Ax(t) + Bu(t)$ が可制御であるとは、任意の初期状態 $x(0) = x_0$ から、適当な有限時刻 $t_f > 0$ まで適当な入力 $u(t) \ (0 \le t \le t_f)$ を加えることで、$x(t_f) = 0$ とできることをいう。

可制御性をチェックするためには、可制御性行列

#### $U_c = [B \quad AB \quad A^2B \quad \cdots \quad A^{n-1}B] \quad (5.58)$

をつくり、それがフルランクとなることを確かめる。つまり、$\text{rank } U_c = n$ となっていることを確認する。ちなみに、$U_c$ が正方行列の場合は、$U_c$ が正則であることを確かめてもかまわない。たとえば、$\det U_c \neq 0$ を調べる。

Python では、ctrb 関数を使って、可制御性行列 $U_c$ を求める。

引数は、$A$ 行列と $B$ 行列で、返り値は $U_c$ となる。

In [ ]:
from control.matlab import ss, ctrb, obsv

In [ ]:
A = [[0, 1], [-4, 5]]
B = [[0], [1]]
C = [1, 0]
P = ss(A, B, C, 0)
print(P)

In [ ]:
# 可制御性行列 Uc を計算
Uc = ctrb(P.A, P.B)

# --- 可制御性行列を表示 ---
print('Uc=\n', Uc)

# --- 行列の行列式を表示 ---
# det(Uc) ≠ 0 なら正則（フルランク）で、可制御である可能性が高い（ただし2次元以上のみ）
print('det(Uc)=', np.linalg.det(Uc))

# --- 可制御性行列のランクを確認 ---
# 状態次元と同じランクであれば、完全可制御
print('rank(Uc)=', np.linalg.matrix_rank(Uc))

### 可観測性

システムの初期値を推定する問題を考える。

初期時刻から有限時刻 $t_f > 0$ までの出力 $y(t)$ および入力 $u(t)$ から初期状態 $x(0)$ を一意に決定できるとき、システムは可観測であるという。システムの可観測性は、第7章で説明するオブザーバの設計で必要となる。

可観測性をチェックするためには、可観測性行列

#### $U_o = \begin{bmatrix} C \\ CA \\ CA^2 \\ \vdots \\ CA^{n-1} \end{bmatrix} \quad (5.59)$

をつくり、それがフルランクとなることを確かめる。つまり、$\text{rank } U_o = n$ となっていることを確認する。ちなみに、$U_o$ が正方行列の場合は、$U_o$ が正則であることを確かめてもかまわない。たとえば、$\det U_o \neq 0$ を調べる。

Python では、obsv 関数を用いて可観測性行列 $U_o$ を求める。

引数は、$A$ 行列と $C$ 行列で、返り値は $U_o$ となる。

In [ ]:
# 可観測性行列 Uo を計算
Uo = obsv(P.A, P.C)

# --- 可観測性行列を表示 ---
print('Uo=\n', Uo)

# --- 行列式を表示 ---
# det(Uo) ≠ 0 であれば正則（正方行列かつフルランク） → 可観測な可能性が高い
print('det(Uo)=', np.linalg.det(Uo))

# --- 可観測性行列のランクを表示 ---
# 状態次元と同じなら、完全可観測
print('rank(Uo)=', np.linalg.matrix_rank(Uo))